In [1]:
import pandas as pd
from sqlalchemy import create_engine
    
engine = create_engine('mysql+pymysql://root@localhost:3306/music_development')
conmy = engine.connect()
    
data_path = '../data/'

In [3]:
sql = '''
SELECT * FROM songs
WHERE id = 5
'''
songs = pd.read_sql(sql, conmy)
songs

,id,rank,name,artist_id,length,location,genre_id,youtube_code,released_date,rating,website
0,5,3,Imagine,4,185,Imagine.mp3,27,rAn-AWXtHv0,1971-10-01,5,0


In [5]:
sql = '''
SELECT * FROM songs
WHERE artist_id = 4
ORDER BY rank 
'''
songs = pd.read_sql(sql, conmy)
songs

,id,rank,name,artist_id,length,location,genre_id,youtube_code,released_date,rating,website
0,5,3,Imagine,4,185,Imagine.mp3,27,rAn-AWXtHv0,1971-10-01,5,0


In [7]:
sql = 'SELECT * FROM lyrics WHERE song_id = 5 '
lyrics = pd.read_sql(sql, conmy)
print(lyrics.content)

0    \r\n\r\nImagine there's no heaven\r\nIt's easy...
Name: content, dtype: object


In [9]:
sql = 'SELECT * FROM artists LIMIT 1'
artists = pd.read_sql(sql, conmy)
artists.dtypes

id             int64
first_name    object
last_name     object
band           int64
image         object
dtype: object

In [10]:
sql = 'SELECT * FROM artists WHERE first_name = "Roxette"'
artist = pd.read_sql(sql, conmy)
artist

,id,first_name,last_name,band,image
0,175,Roxette,,1,Roxette.png


In [5]:
sql = '''
SELECT rank, name, artist_id, released_date FROM songs
'''
songs = pd.read_sql(sql, conmy)
songs.shape

(468, 4)

In [6]:
sql = '''
SELECT id, first_name, last_name
FROM artists
ORDER BY first_name
'''
artists = pd.read_sql(sql, conmy)
artists.shape

(250, 3)

In [7]:
dfm = pd.merge(songs, artists, how='inner', left_on='artist_id',right_on='id')
dfm.dtypes

rank              int64
name             object
artist_id         int64
released_date    object
id                int64
first_name       object
last_name        object
dtype: object

In [8]:
dfm["full_name"] = dfm["first_name"] + ' ' + dfm["last_name"]

In [9]:
columns = ['id','artist_id','first_name','last_name']
dfm = dfm.drop(columns, axis=1)
dfm

,rank,name,released_date,full_name
0,1,Like a Rolling Stone,1965-07-01,Bob Dylan
1,14,Blowin' in the Wind,1963-05-01,Bob Dylan
2,59,The Times They Are A-Changing,1964-01-05,Bob Dylan
3,68,Tangled Up in Blue,1975-01-06,Bob Dylan
4,107,Mr. Tambourine Man,1965-03-05,Bob Dylan
...,...,...,...,...
463,365,Sweet Dreams (Are Made of This),1983-04-11,Eurythmics
464,350,The Harder They Come,1975-03-14,Jimmy Cliff
465,457,The Twist,1960-08-14,Chubby Checker
466,447,Pink Houses,1983-10-14,John Cougar Mellencamp


In [10]:
file_name   = 'songs.csv'
output_file = data_path + file_name
dfm = dfm.set_index(['rank'])
dfm.to_csv(output_file)

In [11]:
columns = ['id','artist_id','first_name','last_name']
dfm.drop(columns,axis=1,inplace=True)
dfm

KeyError: "['id' 'artist_id' 'first_name' 'last_name'] not found in axis"

In [12]:
grouped = dfm.groupby(['full_name'])
grouped

In [13]:
functions = ['count']
result = grouped['full_name'].agg(functions)
result.sort_values(by=['count'],ascending=False)

,count
full_name,
Beatles The,24
Rolling Stones The,15
Bob Dylan,10
Elvis Presley,9
Creedence Clearwater Revival,8
...,...
Howlin’ Wolf,1
Iggy Pop,1
Ike and Tina Turner,1


In [14]:
result

,count
full_name,
50 Cent,1
? and the Mysterians,2
AC/DC,2
Abba,1
Aerosmith,3
...,...
Whitney Houston,2
Who The,5
Willie Nelson,1


In [15]:
dfm[dfm['full_name'] == 'Queen'].sort_values(by=['rank'])

,name,released_date,full_name
rank,,,


In [16]:
dfm.full_name

rank
1                   Bob Dylan
14                  Bob Dylan
59                  Bob Dylan
68                  Bob Dylan
107                 Bob Dylan
                ...          
365               Eurythmics 
350               Jimmy Cliff
457            Chubby Checker
447    John Cougar Mellencamp
423                Don Henley
Name: full_name, Length: 468, dtype: object

In [17]:
sql = '''
SELECT rank, name, artist_id FROM songs 
JOIN lyrics ON lyrics.song_id = songs.id
WHERE website = true AND location IS NOT NULL
'''
songs = pd.read_sql(sql, conmy)
songs.shape

(7, 3)

In [18]:
sql = '''
SELECT id, first_name, last_name, band 
FROM artists
ORDER BY first_name
'''
artists = pd.read_sql(sql, conmy)
artists.shape

(250, 4)

In [19]:
dfm = pd.merge(songs, artists, how='inner', left_on='artist_id',right_on='id')
dfm

,rank,name,artist_id,id,first_name,last_name,band
0,4,What's Going On,5,5,Marvin,Gaye,0
1,134,Won’t Get Fooled Again,11,11,Who,The,1
2,460,My Sweet Lord,191,191,George,Harrison,0
3,7116,You've Got a Friend,140,140,James,Taylor,0
4,285,Ain’t No Sunshine,134,134,Bill,Withers,0
5,7138,If You Could Read My Mind,150,150,Gordon,Lightfoot,0
6,7172,(Where Do I Begin?) Love Story,223,223,Andy,Williams,0


In [20]:
file_name    = 'music.csv'
output_file = data_path + file_name
df_out = dfm.set_index(['rank'])
df_out.to_csv(output_file, header = None)

In [21]:
sql = '''
SELECT * 
FROM lyrics
WHERE song_id NOT IN
(SELECT id FROM songs)
'''
lyric = pd.read_sql(sql, conmy)
lyric

,id,song_id,content,created_at,updated_at


In [22]:
sql = '''
SELECT * 
FROM songs
WHERE rank = 434
'''
song = pd.read_sql(sql, conmy)
song

,id,rank,name,artist_id,length,location,genre_id,youtube_code,released_date,rating,website
0,413,434,Smoke on the Water,123,300,Smoke_on_the_Water.mp3,27,ikGyZh0VbPQ,1973-05-05,0,0


In [23]:
sql = '''
SELECT * 
FROM artists
WHERE first_name = 'Deep Purple'
'''
artist = pd.read_sql(sql, conmy)
artist

,id,first_name,last_name,band,image
0,123,Deep Purple,,1,Deep_Purple.jpg


In [24]:
sql = '''
UPDATE songs
SET artist_id = 123
WHERE rank = 434
'''
rp = conmy.execute(sql)
rp.rowcount

1

In [25]:
sql = '''
UPDATE lyrics
SET song_id = 413
WHERE id  = 216
'''
rp = conmy.execute(sql)
rp.rowcount

1

In [26]:
sql = '''
DELETE FROM songs WHERE rank = 501
'''
rp = conmy.execute(sql)
rp.rowcount

1

In [27]:
sql = '''
UPDATE songs
SET website = false
WHERE website = true
'''
rp = conmy.execute(sql)
rp.rowcount

7

### New music database Melody

In [28]:
col_names   = ['year','rank','title','artist_name']
songs = pd.read_csv('c:\\aab\\data\\Billboard.csv', sep=';', header=None, index_col=None, names=col_names)
songs.shape    

(5800, 4)

In [29]:
songs.head()

,year,rank,title,artist_name
0,1960,1,Theme from A Summer Place,Percy Faith
1,1960,2,He'll Have to Go,Jim Reeves
2,1960,3,Cathy's Clown,The Everly Brothers
3,1960,4,Running Bear,Johnny Preston
4,1960,5,Teen Angel,Mark Dinning


In [30]:
grouped = songs.groupby(['artist_name'])
grouped

In [31]:
functions = ['count']
result = grouped['artist_name'].agg(functions)

In [32]:
sorted = result.sort_values(by=['count'],ascending=[False])
sorted

,count
artist_name,
Madonna,35
Elton John,26
The Beatles,26
Janet Jackson,24
Mariah Carey,24
...,...
"DJ Khaled featuring Justin Bieber, Quavo, Chance the Rapper and Lil Wayne",1
Lou Bega,1
DJ Khaled featuring Rihanna and Bryson Tiller,1


In [33]:
sorted.to_csv('c:\\aab\\data\\artists.csv', header=None, sep=';')